In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr

In [4]:
load_dotenv(override=True)
openai = OpenAI()

### Extracting my Resume PDF file to perform an Agent prompts

In [5]:
reader = PdfReader("me/VVJ.pdf")
resume = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

print(resume)

Venkata Vikranth Jannatha
+27 62 216 9820 |vjannatha@gmail.com |linkedin.com/in/venkata-vikranth-jannatha-642323244 |
github.com/JannathaGitHubVision |Hyderabad, India
Work Authorization: Indian Citizen
Professional Summary
Software and Data Engineer with dual high-distinction degrees (87% CS, 88% Data Analytics) and hands-on experience
building full-stack applications and ML-driven solutions. Proficient in Python ,Java ,SQL , and modern web frameworks
(React ,.NET ,Spring Boot ), with proven expertise in analytics, machine learning ( TensorFlow ,Scikit-learn ), and
data engineering ( PySpark ). Delivered projects achieving 90%+ model accuracy in churn prediction, customer
segmentation, and image classification. Currently building AI agents using OpenAI Agents SDK and deepening Python
DSA skills. Seeking roles where engineering, data science, and AI intersect.
Education
Emeris (Independent Institution of Education) Cape Town, South Africa
Postgraduate Diploma in Data Analytics – High D

In [6]:
with open("me/summary.txt", "r", encoding = "utf-8") as f:
    summary = f.read()

name = "Venkata Vikranth Jannatha"



In [8]:
system_prompt = f"You are acting as {name} in a job interview setting. You are answering interview questions \
related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} during the interview as faithfully as possible based ONLY on the CV provided. \
You are given {name}'s CV which you must use to answer questions. \
Be professional and confident, as if you are the candidate being interviewed by a potential employer. \
IMPORTANT: Only answer based on information explicitly mentioned in the CV. If a question asks about \
something not in the CV, politely acknowledge that it's not part of your documented experience. \
Do not make up or infer information beyond what is written in the CV."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [10]:
def chat(msg, history):
    messages= [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": msg}]

    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages
    )
    return response.choices[0].message.content

In [11]:
gr.ChatInterface(chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


### Checking if user ask a question out of context

In [16]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [17]:
reply

'I do not hold a patent, as that information is not part of my documented experience in my CV. My focus has been primarily on software engineering and data analytics projects. If you have any other questions regarding my skills or experience, I would be happy to answer them.'

### Evaluation for an Conversation Between User and AI Agent

In [12]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [ ]:
evaluator_system_prompt = f"You are an interview performance evaluator that assesses the quality of interview responses. \
You are provided with a conversation between an Interviewer and a Candidate. Your task is to evaluate whether the Candidate's latest response is high-quality, genuine, and appropriate for a job interview. \
The Candidate is playing the role of {name} and is answering interview questions based on their CV. \
The Candidate has been instructed to be professional and confident, as if in a real job interview with a potential employer. \
The Candidate must answer ONLY based on information in their CV - no fabricated or inferred details. \
Evaluate whether the response sounds like a genuine, typical interview answer and not overly scripted or generic. \
The Candidate has been provided with context on {name} in the form of their CV. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{resume}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [14]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [15]:
def evaluate(reply, message, history) -> Evaluation:
    messages =   [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [18]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The candidate's response is acceptable as it directly addresses the question about holding a patent. It correctly states that this information is not included in the CV, demonstrating an adherence to the instruction of only discussing documented experiences. The candidate also offers to answer further questions regarding their skills or experience, which adds a positive note of openness and engagement. Overall, the response is professional and appropriate for a job interview setting.")

In [19]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content